In [1]:
'''
계산한 단기 지표들을 데이터베이스에서 꺼냄
'''
import pandas as pd
import sqlite3

db_path = 'Database/S&P500.db'
conn = sqlite3.connect(db_path)

query = "SELECT gics_sector FROM stock;"
stocks = pd.read_sql_query(query, conn)
sectors = stocks['gics_sector'].unique()
print(sectors)

query = "SELECT * FROM daily_stock_indicator;"
daily_stock_indicator = pd.read_sql_query(query, conn)
display(daily_stock_indicator.head())

conn.close()

['Industrials' 'Health Care' 'Information Technology' 'Utilities'
 'Financials' 'Materials' 'Consumer Discretionary' 'Real Estate'
 'Communication Services' 'Consumer Staples' 'Energy']


,symbol,gics_sector,date,adj_close,close,high,low,open,volume,sma_5,sma_10,sma_20,ema_5,ema_10,ema_20,macd,macd_signal,macd_diff,adx,adx_pos,adx_neg,parabolic_sar,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,rsi_14,stochastic_k,stochastic_d,tsi,roc_12,cmo,williams_%r,cci_10,cci_20,bollinger_upper,bollinger_middle,bollinger_lower,atr_14,keltner_hband,...,pivot_point_2,doji,hammer,inverted_hammer,shooting_star,morning_star,triple_top,triple_bottom,red_marubozu,green_marubozu,three_black_crows,three_white_soldiers,change_1td,change_2td,change_3td,change_4td,change_5td,change_6td,change_7td,change_8td,change_9td,change_10td,change_11td,change_12td,change_13td,change_14td,change_15td,change_16td,change_17td,change_18td,change_19td,change_20td,change_21td,change_22td,change_23td,change_24td,change_25td,change_26td,change_27td,change_28td
0,MMM,Industrials,2010-01-04 00:00:00,43.783875,69.414719,69.774246,69.122070,69.473244,3640265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,69.414719,NaN,69.448158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,70.089188,...,69.446070,1,0,0,0,0,0,0,0,0,0,0,-0.626357,0.782935,0.855212,1.565881,1.156345,1.240657,0.915432,0.578172,0.421583,2.529509,2.047695,-0.385456,-1.854979,-1.433396,-1.674297,-0.867269,-2.734283,-3.047462,-3.107693,-3.119739,-1.830886,-4.589262,-5.396289,-6.612865,-5.179480,-5.299931,-3.312456,-4.625400
1,MMM,Industrials,2010-01-05 00:00:00,43.509628,68.979935,69.590302,68.311035,69.230766,3405012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,68.979935,NaN,69.042641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,70.164439,...,69.028009,0,0,0,0,0,0,0,0,0,0,0,1.418175,1.490908,2.206056,1.793938,1.878782,1.551507,1.212121,1.054545,3.175758,2.690907,0.242420,-1.236366,-0.812125,-1.054545,-0.242431,-2.121213,-2.436365,-2.496976,-2.509098,-1.212121,-3.987883,-4.799998,-6.024241,-4.581822,-4.703032,-2.703029,-4.024250,-2.460609
2,MMM,Industrials,2010-01-06 00:00:00,44.126682,69.958191,70.735786,69.824417,70.133781,6301126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,68.311035,NaN,69.523411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,70.471015,...,70.163044,0,0,1,0,0,0,0,0,0,1,0,0.071716,0.776864,0.370509,0.454166,0.131468,-0.203172,-0.358545,1.733006,1.254935,-1.159314,-2.617421,-2.199113,-2.438143,-1.637385,-3.489895,-3.800640,-3.860403,-3.872356,-2.593516,-5.330463,-6.131221,-7.338346,-5.916096,-6.035611,-4.063576,-5.366321,-3.824546,-4.218948
3,MMM,Industrials,2010-01-07 00:00:00,44.158318,70.008362,70.033447,68.662209,69.665550,5346240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,68.311035,NaN,69.523411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,70.588072,...,69.592392,0,1,0,0,1,0,0,0,0,0,0,0.704643,0.298579,0.382176,0.059709,-0.274691,-0.429952,1.660100,1.182371,-1.230147,-2.687210,-2.269201,-2.508060,-1.707876,-3.559058,-3.869581,-3.929301,-3.941245,-2.663321,-5.398307,-6.198491,-7.404751,-5.983521,-6.102950,-4.132328,-5.434139,-3.893469,-4.287589,-3.129106
4,MMM,Industrials,2010-01-08 00:00:00,44.469475,70.501671,70.501671,69.648827,69.974915,4073337,69.772575,NaN,NaN,69.94653,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,68.408025,NaN,69.523411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,70.684505,...,70.156771,0,0,0,0,1,1,1,0,0,0,0,-0.403223,-0.320211,-0.640421,-0.972481,-1.126656,0.948771,0.474386,-1.921252,-3.368119,-2.953036,-3.190223,-2.395638,-4.233867,-4.542217,-4.601520,-4.613380,-3.344398,-6.060247,-6.854832,-8.052651,-6.641366,-6.759960,-4.803126,-6.095828,-4.565938,-4.957301,-3.806924,-3.320677


In [2]:
'''
데이터 전처리
'''
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

data_all = daily_stock_indicator.copy()


# 결측치가 존재하는 행 제거 (대략 1.8%)
data_all = data_all.dropna()


# 날짜 컬럼 타입변경
data_all['date'] = pd.to_datetime(data_all['date'])


# n일 뒤 변화율만 남김
after_td = 1
data_all = data_all.drop(columns=[f'change_{i}td' for i in range(1, 7 * 4 + 1) if i != after_td])
data_all = data_all.rename(columns={f'change_{after_td}td':'change'})


# 데이터 정규화
mmscaler = MinMaxScaler(feature_range=(0, 1))
data_all = data_all.set_index(['date', 'symbol', 'gics_sector'])
tmp1 = data_all.loc[ : ,  :'three_white_soldiers']
tmp2 = data_all.iloc[ : , -1]
data_all = pd.concat([pd.DataFrame(data=mmscaler.fit_transform(tmp1), index=data_all.index, columns=data_all.loc[ : , :'three_white_soldiers'].columns), tmp2], axis=1)
data_all = data_all.reset_index()


display(data_all.head())

,date,symbol,gics_sector,adj_close,close,high,low,open,volume,sma_5,sma_10,sma_20,ema_5,ema_10,ema_20,macd,macd_signal,macd_diff,adx,adx_pos,adx_neg,parabolic_sar,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,rsi_14,stochastic_k,stochastic_d,tsi,roc_12,cmo,williams_%r,cci_10,cci_20,bollinger_upper,bollinger_middle,bollinger_lower,atr_14,keltner_hband,keltner_lband,donchian_hband,donchian_lband,obv,mfi_14,volume_oscillator,adl,vwap_14,pivot_point_1,pivot_point_2,doji,hammer,inverted_hammer,shooting_star,morning_star,triple_top,triple_bottom,red_marubozu,green_marubozu,three_black_crows,three_white_soldiers,change
0,2010-02-26,MMM,Industrials,0.004267,0.006730,0.006718,0.006731,0.006703,0.001044,0.006798,0.006901,0.006885,0.006807,0.006861,0.006951,0.615300,0.577717,0.639274,0.178453,0.186587,0.223297,0.006834,0.006933,0.007334,0.006995,0.006873,0.471976,0.643016,0.606799,0.427517,0.423592,0.573516,0.643016,0.440114,0.499404,0.006839,0.006885,0.010812,0.003941,0.006873,0.006888,0.006845,0.006894,0.089873,0.489782,0.172736,0.104758,0.006855,0.006726,0.006720,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.536488
1,2010-03-01,MMM,Industrials,0.004290,0.006767,0.006770,0.006838,0.006773,0.000824,0.006796,0.006913,0.006885,0.006821,0.006868,0.006952,0.615381,0.577786,0.639333,0.162930,0.210339,0.214227,0.006828,0.006896,0.007334,0.006921,0.006873,0.496328,0.694299,0.629807,0.434668,0.426853,0.647059,0.694299,0.560307,0.550588,0.006840,0.006885,0.010812,0.003814,0.006868,0.006892,0.006835,0.006894,0.089897,0.557064,0.172738,0.104750,0.006874,0.006791,0.006786,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.682556
2,2010-03-02,MMM,Industrials,0.004319,0.006813,0.006806,0.006857,0.006787,0.001090,0.006821,0.006918,0.006888,0.006845,0.006882,0.006958,0.615502,0.577872,0.639462,0.150690,0.223969,0.205241,0.006823,0.006896,0.007334,0.006921,0.006873,0.526425,0.832447,0.723254,0.445031,0.420000,0.623844,0.832447,0.616667,0.572548,0.006848,0.006888,0.010810,0.003696,0.006870,0.006895,0.006835,0.006894,0.089928,0.550195,0.172743,0.104766,0.006889,0.006825,0.006816,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.160237
3,2010-03-03,MMM,Industrials,0.004312,0.006802,0.006831,0.006881,0.006823,0.000924,0.006836,0.006926,0.006891,0.006858,0.006891,0.006962,0.615589,0.577961,0.639496,0.142067,0.230205,0.196152,0.006577,0.006895,0.007334,0.006918,0.006873,0.518557,0.794038,0.773595,0.452690,0.425916,0.621926,0.794038,0.619700,0.575891,0.006854,0.006891,0.010809,0.003590,0.006871,0.006901,0.006835,0.006894,0.089902,0.602315,0.172742,0.104743,0.006901,0.006838,0.006834,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.024693
4,2010-03-04,MMM,Industrials,0.004313,0.006804,0.006804,0.006856,0.006827,0.000849,0.006858,0.006925,0.006889,0.006867,0.006899,0.006966,0.615662,0.578051,0.639496,0.130977,0.220152,0.203553,0.006582,0.006895,0.007334,0.006918,0.006873,0.519749,0.799458,0.808648,0.459234,0.418130,0.546125,0.799458,0.577113,0.564086,0.006847,0.006889,0.010812,0.003485,0.006867,0.006905,0.006835,0.006894,0.089926,0.522508,0.172742,0.104747,0.006908,0.006821,0.006823,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.752647


In [3]:
'''
데이터 선택
'''
# all, sector
selector = 'all'

dps = {} # data per sector
if selector == 'all':
    dps['all'] = data_all.copy()
elif selector == 'sector':
    for sector in sectors:
        dps[sector] = data_all.loc[data_all['gics_sector'] == sector].copy()

In [5]:
'''
모델링 및 검증 : XGBoost
'''
import pandas as pd
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for sector, data in dps.items():    
    # 업종 컬럼 삭제
    data = data.drop(columns=['gics_sector'])

    # 종목 컬럼 삭제 or 종목이름 원핫인코딩
    data = data.drop(columns=['symbol'])

    # 학습, 검증 데이터 분리
    x_train = data.loc[data['date'].between('2001-01-01', '2022-01-01', inclusive='left') | data['date'].between('2023-01-01', '2034-01-01', inclusive='left'), :'three_white_soldiers']
    x_test = data.loc[data['date'].between('2022-01-01', '2023-01-01', inclusive='left'), :'three_white_soldiers']

    y_train_r = data.loc[data['date'].between('2001-01-01', '2022-01-01', inclusive='left') | data['date'].between('2023-01-01', '2034-01-01', inclusive='left'), 'change']
    y_test_r = data.loc[data['date'].between('2022-01-01', '2023-01-01', inclusive='left'), 'change']

    y_train_c = y_train_r > 0
    y_test_c = y_test_r > 0


    # 학습, 검증 데이터의 날짜 처리
    # # 연, 월, 일 분리하여 생성 (컬럼 선택)
    # x_train['year'] = x_train['date'].dt.year
    # x_train['month'] = x_train['date'].dt.month
    # x_train['day'] = x_train['date'].dt.day

    # x_val['year'] = x_val['date'].dt.year
    # x_val['month'] = x_val['date'].dt.month
    # x_val['day'] = x_val['date'].dt.day

    # x_test['year'] = x_test['date'].dt.year
    # x_test['month'] = x_test['date'].dt.month
    # x_test['day'] = x_test['date'].dt.day


    # 날짜 컬럼 삭제
    x_train = x_train.drop(columns=['date'])
    # x_val = x_val.drop(columns=['date'])    
    x_test = x_test.drop(columns=['date'])


    start = time.time()
    
    # 모델 학습
    # Classifier
    # model_c = XGBClassifier(
    #     n_estimators = 10, 
    #     learning_rate = 0.1, 
    #     max_depth = 30, 
    #     min_child_weight = 5,
    #     gamma = 0.1,
    #     subsample = 0.8, 
    #     eval_metric = 'logloss',
    #     device = 'cuda'
    # )
    # model_c.fit(x_train, y_train_c, verbose=False)
    # pred_c = model_c.predict(x_test)


    
    # Regressor
    model_r = XGBRegressor(
        n_estimators = 100, 
        learning_rate = 0.1, 
        max_depth = 30, 
        min_child_weight = 5,
        gamma = 0.1,
        subsample = 0.8, 
        eval_metric = 'logloss',
        device = 'cuda'
    )
    model_r.fit(x_train, y_train_r, verbose=False)
    pred_r = model_r.predict(x_test)
    pred_c = pd.Series(pred_r) > 0
    
    end = time.time()
    print(f"Learning Time : {end - start:.5f} sec")


    # 점수화
    acs = accuracy_score(y_test_c, pred_c)
    prs = precision_score(y_test_c, pred_c)
    res = recall_score(y_test_c, pred_c)
    f1s = f1_score(y_test_c, pred_c)
    print('SECTOR : ', sector)
    print('accuracy_score : ', acs)
    print('precision_score : ', prs)
    print('recall_score : ', res)
    print('f1_score : ', f1s)
    accuracy_scores.append(acs)
    precision_scores.append(prs)
    recall_scores.append(res)
    f1_scores.append(f1s)
    print()

    print("Train Values : ")
    display(y_train_c.value_counts())
    print()

    # print("Validation Values : ")
    # display(y_val_c.value_counts())
    # print()

    print("Test Values : ")
    display(y_test_c.value_counts())
    print()

    print("Predict Values : ")
    display(pd.DataFrame({'predict' : pred_c}).value_counts())
    print()

print('accuracy_score mean : ', sum(accuracy_scores) / len(accuracy_scores))
print('precision_score mean : ', sum(precision_scores) / len(precision_scores))
print('recall_score mean : ', sum(recall_scores) / len(recall_scores))
print('f1_score mean : ', sum(f1_scores) / len(f1_scores))

Learning Time : 372.83490 sec
SECTOR :  all
accuracy_score :  0.5062607909896245
precision_score :  0.5018351143513432
recall_score :  0.6537379453903823
f1_score :  0.5678025354002613

Train Values : 


change
True     878760
False    744708
Name: count, dtype: int64


Test Values : 


change
False    62455
True     61491
Name: count, dtype: int64


Predict Values : 


predict
True       80104
False      43842
Name: count, dtype: int64


accuracy_score mean :  0.5062607909896245
precision_score mean :  0.5018351143513432
recall_score mean :  0.6537379453903823
f1_score mean :  0.5678025354002613
